In [1]:
import pandas as pd
import pymysql

# Establish connection to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='DIsaster@2003', database='REDBUS')

# Read the CSV file
df = pd.read_csv("C:/Project-Redbus-main/redbus_all_data.csv")
df = df.dropna().iloc[:, 1:]

# Ensure correct column names
print("Columns in DataFrame:", df.columns)

# Format the DataFrame columns
try:
    df['Departure_Time'] = pd.to_datetime(df['Departure_Time'], format='%H:%M').dt.time
    df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'], format='%H:%M').dt.time
except Exception as e:
    print(f"Error parsing date columns: {e}")

# Clean 'Seat_Availability' column and handle NaN values
if 'Seat_Availability' in df.columns:
    # Extract numerical part and convert to int
    df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)').fillna(0).astype(int)
else:
    print("Warning: 'Seat_Availability' column not found")

# Handle NaN values in 'id' column
if 'id' in df.columns:
    df['id'] = df['id'].astype(int)
else:
    print("Warning: 'id' column not found")

# Ensure 'Rating' and 'Fare' columns are properly formatted
df['Rating'] = df['Rating'].astype(float)
df['Fare'] = df['Fare'].astype(float)

# Map pandas dtypes to MySQL dtypes
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INT',
    'int32': 'INT',
    'float64': 'FLOAT',
    'datetime64[ns]': 'DATETIME',
    'datetime.time': 'TIME'
}

# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

# Generate the formatted string for creating the table with escaped column names
a = ",".join(f"`{i}` {dtype_mapping[str(j)]}" for i, j in zip(df.columns, df.dtypes))

table_name = "redbus"

# Create cursor object
cursor = myconnection.cursor()

# Drop table if it exists (optional)
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

# Create table
create_table_query = f"CREATE TABLE {table_name} ({a})"
print("Create Table Query:", create_table_query)  # Print the query to debug
cursor.execute(create_table_query)

# Insert data into the table
insert_query = f"INSERT INTO {table_name} VALUES ({','.join(['%s'] * len(df.columns))})"

# Execute the insert queries
for row in df.itertuples(index=False, name=None):
    cursor.execute(insert_query, row)

# Commit the changes
myconnection.commit()

# Close the cursor and connection
cursor.close()
myconnection.close()

<>:24: SyntaxWarning: invalid escape sequence '\d'
<>:24: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jakis\AppData\Local\Temp\ipykernel_13532\1755050967.py:24: SyntaxWarning: invalid escape sequence '\d'
  df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)').fillna(0).astype(int)


Columns in DataFrame: Index(['Bus_name', 'Bus_Type', 'Departure_Time', 'Arrival_Time', 'Duration',
       'Fare', 'Rating', 'Seat_Availability', 'Bus_Link', 'Bus_Route'],
      dtype='object')
Create Table Query: CREATE TABLE redbus (`Bus_name` TEXT,`Bus_Type` TEXT,`Departure_Time` TEXT,`Arrival_Time` TEXT,`Duration` TEXT,`Fare` FLOAT,`Rating` FLOAT,`Seat_Availability` INT,`Bus_Link` TEXT,`Bus_Route` TEXT)


,Bus_name,Bus_Type,Departure_Time,Arrival_Time,Duration,Fare,Rating,Seat_Availability,Bus_Link,Bus_Route
37,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,16:00:00,19:45:00,03h 45m,170.0,2.6,19,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
38,Shirdi Xpress,A/C Sleeper (2+1),08:15:00,11:30:00,03h 15m,819.0,3.4,1,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
39,Humsafar Travels,A/C Sleeper (2+1),04:30:00,09:00:00,04h 30m,1650.0,1.3,17,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
49,Ashoka Travels,A/C Sleeper (2+1),08:30:00,14:50:00,06h 20m,400.0,3.5,18,https://www.redbus.in/bus-tickets/goa-to-kolha...,Goa to Kolhapur(Maharashtra)
61,Humsafar Travels,A/C Sleeper (2+1),01:00:00,09:00:00,08h 00m,1850.0,4.0,20,https://www.redbus.in/bus-tickets/sangola-to-goa,Sangola (Solapur) to Goa
...,...,...,...,...,...,...,...,...,...,...
855,Express Line,Volvo A/C Seater (2+2),11:10:00,15:20:00,04h 10m,399.0,4.2,15,https://www.redbus.in/bus-tickets/mandarmani-t...,Mandarmani to Kolkata
856,Express Line,Volvo 9600 Multi Axle Semi-Sleeper (2+2),08:30:00,12:10:00,03h 40m,699.0,4.7,7,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mandarmani
857,Greenline,Volvo A/C Seater (2+2),06:30:00,10:50:00,04h 20m,750.0,4.7,5,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mandarmani
858,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),07:15:00,11:00:00,03h 45m,650.0,4.4,2,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mandarmani
